In [5]:
import re
from pathlib import Path
from requests_html import HTMLSession
sess = HTMLSession()

In [2]:
url = "http://46.166.167.16/forums/pemaksaan.44/"


In [3]:
r = sess.get(url)

In [4]:
r.html.links

{'/',
 '/forums/pemaksaan.44/#navigation',
 '/help/advertise/',
 '/help/dmca/',
 '/help/statistic/',
 '/misc/contact',
 'account/dismiss-notice?notice_id=15',
 'find-new/posts',
 'forums/-/index.rss',
 'forums/pemaksaan.44/',
 'forums/pemaksaan.44/?direction=asc',
 'forums/pemaksaan.44/?order=post_date',
 'forums/pemaksaan.44/?order=reply_count',
 'forums/pemaksaan.44/?order=title',
 'forums/pemaksaan.44/?order=view_count',
 'forums/pemaksaan.44/?prefix_id=34',
 'forums/pemaksaan.44/?prefix_id=35',
 'forums/pemaksaan.44/page-19',
 'forums/pemaksaan.44/page-2',
 'forums/pemaksaan.44/page-3',
 'forums/pemaksaan.44/page-4',
 'forums/pemaksaan.44/page-5',
 'forums/pemaksaan.44/page-6',
 'help/',
 'help/privacy-policy',
 'help/terms',
 'http://46.166.167.16/',
 'http://46.166.167.16/#khas-semprot.6',
 'http://46.166.167.16/forums/cerita.42/',
 'http://tinyurl.com/cintasemprot',
 'http://tinyurl.com/ya7zpe8t',
 'http://www.facebook.com/semprot',
 'http://xf.themehouse.com/',
 'https://twitte

In [53]:
PATTERN = r'.\d{7}/'
PATTERN

'.\\d{7}/'

In [89]:
PATTERN = r'\w+.\d{7}'
PATTERN

'\\w+.\\d{7}[/]'

In [127]:
PATTERN = r'[\d]{3,12}[/]$'
PATTERN

'[\\d]{3,12}[/]$'

In [128]:
[u for u in list(r.html.links) if re.search(PATTERN, u) and 'threads' in u]

['threads/memperbudak-hilda-si-tetangga-yang-bohay.1270902/',
 'threads/di-sebuah-panti-pijat-by-mascoro.1203719/',
 'threads/karyawan-bodoh-diperkosa.1280049/',
 'threads/tania-budak-seks-dosen-pembimbing.1261325/',
 'threads/bem-badan-enak-mahasiswi.1219898/',
 'threads/jalanin-aja-dulu.1279653/',
 'threads/terpaksa-aku-harus-begini.1246537/',
 'threads/my-rape-my-adventure.1246887/',
 'threads/aku-dosen-beruntung-mahasiswi-jilbab-copas.1261332/',
 'threads/penderitaan-fenny.1209802/',
 'threads/tante-nathalie-dikerjain-tukang-bully-anaknya.1284753/',
 'threads/judul-cerita-panas-pemerkosa-update.973838/',
 'threads/predator.1071030/',
 'threads/dara-the-innocent.1283682/',
 'threads/nikmatilah-istriku-yang-cantik-dan-polos-ini.1219536/',
 'threads/rules-cerita-panas-pemerkosa-wajib-baca-sebelum-posting.972659/',
 'threads/perkosaan-rame2-dalam-permainan.1246705/',
 'threads/rumah-kost-no-39-sherly.1067026/',
 'threads/badan-enak-mahasiswi-remake-by-marcioz-tamat.1268774/',
 'threads

In [46]:
u

'threads/gara-gara-telat-bangun-real-story.1249547/page-17'

In [43]:
re.search(PATTERN, u) == True

False

In [37]:
if re.search(PATTERN, u): print('WOW')

WOW


In [33]:
re.compile(PATTERN) in u

TypeError: 'in <string>' requires string as left operand, not _sre.SRE_Pattern

In [20]:
sub('12345678')

''

#### Site to mine: https://www.mindat.org/photos/555/
#### Sample jpg: https://www.mindat.org/photos/555/01/0555010001421587987.jpg
#### HTTP Get Request has to have a referer to parent directory

In [2]:
import os
import tqdm
import requests
from requests_html import HTMLSession

Helper Functions

In [3]:
def printf(s):
    if DEBUG: print(s)

def loc_path(url): 
    """Local path without https url"""
    return url[len(ROOT_URL):] if IS_FOLDERS else url.split('/')[-1]

def mkdir_loc(url):
    """Create sub-folders in local directory."""
    if IS_FOLDERS and not os.path.exists(loc_path(url)):
        os.mkdir(loc_path(url))
        printf(f"Created folder at {loc_path(url)}")

def try_http_get(url, **kwargs):
    try:
        headers = kwargs.get('headers')
        r = sess.get(url, headers=headers)
        return r
    except requests.exceptions.RequestException as e:
        print(e, url)
        return None

In [4]:
### INPUTS ###
IS_FOLDERS = True
DEBUG = True
ROOT_URL = 'https://www.mindat.org/'
PARENT_DIR = 'photos/'
### END INPUTS ###

In [5]:
# create HTML session
sess = HTMLSession()

In [6]:
# first parent
p1 = ROOT_URL+PARENT_DIR
mkdir_loc(p1)
r = try_http_get(p1)
printf(f"parent url: {p1}, status code: {r.status_code}")

# first child
c1s = [p1+'{:03}/'.format(i) for i in range(1000)]
for c1 in tqdm.tqdm(c1s):
    mkdir_loc(c1)
    r = try_http_get(c1, headers={'referer': '/'.join(c1.split('/')[:-1])})
    if r is None: continue
    printf(f"first child url: {c1}, status code: {r.status_code}")

    # second child
    c2s = sorted([c1+u for u in list(r.html.links) if len(u) == 3])
    for c2 in tqdm.tqdm(c2s):
        mkdir_loc(c2)
        r = try_http_get(c2, headers={'referer': '/'.join(c2.split('/')[:-1])})
        if r is None: continue
        printf(f"second child url: {c2}, status code: {r.status_code}")
    
        # third child
        c3s = [c2+u for u in list(r.html.links) if '.jpg' in u or '.png' in u]
        for c3 in c3s:
            # get JPG HTML response
            r = try_http_get(c3, headers={'referer': '/'.join(c3.split('/')[:-1])})
            if r is None: continue
            printf(f"third child url: {c3}, status code: {r.status_code}")
            # save image
            if r.status_code == 200 and r.content != b'':
                jpg_path = loc_path(c3)
                with open(jpg_path, 'wb') as f:
                    f.write(r.html.raw_html)



parent url: https://www.mindat.org/photos/, status code: 200


  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

first child url: https://www.mindat.org/photos/000/, status code: 200



  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

second child url: https://www.mindat.org/photos/000/00/, status code: 200
third child url: https://www.mindat.org/photos/000/00/00000020014378957949659.jpg, status code: 200
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)) https://www.mindat.org/photos/000/00/00000020014460934567596.jpg
third child url: https://www.mindat.org/photos/000/00/0000007001315582593.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/00/0000005001177189069.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/00/0000002001318303526.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/00/00000040014610169264918.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/00/0000009001273905760.jpg, status code: 200


...


  3%|██▍                                                                               | 3/100 [00:10<05:52,  3.64s/it]

second child url: https://www.mindat.org/photos/000/03/, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000031001254277428.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000036001219518888.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000035001201636084.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000036001420862967.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000030001324990882.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000039001347316317.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000031001396369388.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000036001431845590.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/03/0000034001228415823.jpg, status code: 200



  4%|███▎                                                                              | 4/100 [00:13<05:32,  3.46s/it]

second child url: https://www.mindat.org/photos/000/04/, status code: 200
third child url: https://www.mindat.org/photos/000/04/0000044001019964891.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/04/00000410015190648377627.jpg, status code: 200
third child url: https://www.mindat.org/photos/000/04/00000420014894491792096.jpg, status code: 200


KeyboardInterrupt: 